In [1]:
from bs4 import BeautifulSoup
import urllib.request
import os

from lxml import etree
import re
import requests
import string
import json
import random
import time

In [2]:
class WebSpider:
    def __init__(self):
        """
        初始化，使用百度浏览器
        """
        # 要请求的网络地址：搜索页
        self.base_url = "http://www.baidu.com/s?wd="# 是http，不是https。因为https是网络的安全协议模式，需要有证书才能返回响应，在没有证书的情况下是不会返回结果的。
        # 查找的关键词
        self.search_word = ""
        # 构造url
        self.url = ""
        # 每个关键词需要多少页
        self.pagenum = -1
        # 该搜索引擎下，该关键词所需要页数内的所有url
        self.pageurl = []
        #请求头
        '''
        self.headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE',
        'accept':'image/webp,image/apng,image/*,*/*;q=0.8',
        'accept-encoding':'gzip, deflate, br',
        'accept-language':'zh-CN,zh;q=0.9',
        'cookie':'BIDUPSID=2806AB9D3577A20F1291141A40A433B6; PSTM=1539695730; BDUSS=BaRE5GfnpaYUlabmhLQ3hWT3B-T2d3QWdtTWNWbGprbmRvT2tXVjVkRFVXNmxmSVFBQUFBJCQAAAAAAAAAAAEAAAAvjQhsRXVydXM1MTEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAANTOgV~UzoFfaT; __yjs_duid=1_bcfa109e4f513a1d5c06db1433f59cce1619712632146; MCITY=-358%3A; BAIDUID=1309F011CED276B13DB5CA747DB72D3C:FG=1; BDORZ=B490B5EBF6F3CD402E515D22BCDA1598; BDSFRCVID=JFuOJexroG0zP23H6THy2Z9Na2KK0gOTDYrEOwXPsp3LGJLVgUSbEG0PtOUBxFP-oxShogKK0gOTH6KF_2uxOjjg8UtVJeC6EG0Ptf8g0M5; H_BDCLCKID_SF=tRk8oIL2fIvbfP0k-J32h-QH-UnLetJXfa5qLl7F54nKDp0x2l0KDUvB3-7rJqI8BDCHahkM5h7xsMT4QfrB0J0WWlJBLpomQDvPbboN3KJmfh7aLR7P0Kue2bb32-biWbRM2MbdJqvP_IoG2Mn8M4bb3qOpBtQmJeTxoUJ25DnJhhCGe6LbejOBjaAs5JtXKD600PK8Kb7VbpcD5fnkbJkXhPtjhq8Ht2-q-pnXtfFB8RD4yU42bU47QbrH0xRfyNReQIO13hcdSR3a0P5pQT8r5-nP3JvlLPjyXUOSab3vOIJTXpO1jxPzBN5thURB2DkO-4bCWJ5TMl5jDh3Mb6ksD-FtqtJHKbDtVI_bfxK; delPer=0; PSINO=5; H_PS_PSSID=35105_31253_34584_34505_35317_26350_35147_35301; BA_HECTOR=000g2g8k80a02k0gl11gqf03a0r'
        }
        '''
        self.headers = [{
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.43',
        'accept':'*/*',
        'accept-encoding':'gzip, deflate, br',
        'accept-language':'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
        'cookie':'BIDUPSID=CCF21A67A067436BA760FEAFE49927C7; PSTM=1601860438; MCITY=-289:; BDUSS=2g0TThYOGFKMVdDdEhSeDJ3T05IQWZzaXExYVMxOHlZNXN2akRWbWxsRH5GdUJnRVFBQUFBJCQAAAAAAAAAAAEAAAAd8kSvw8jMq8bmMTk4MTgwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAP-JuGD~ibhgR; BDUSS_BFESS=2g0TThYOGFKMVdDdEhSeDJ3T05IQWZzaXExYVMxOHlZNXN2akRWbWxsRH5GdUJnRVFBQUFBJCQAAAAAAAAAAAEAAAAd8kSvw8jMq8bmMTk4MTgwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAP-JuGD~ibhgR; __yjs_duid=1_dfcd058e62964219bba57451e64b682b1630648087098; BAIDUID=2A40EB7932004A91EADF3B5E797A87C8:FG=1; BAIDUID_BFESS=2A40EB7932004A91EADF3B5E797A87C8:FG=1; H_PS_PSSID=35360_35106_31253_34584_34517_35245_35320_26350_35301; BA_HECTOR=0la40l84818105059s1gqrqul0q'
        },
        {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
        'accept':'application/json, text/javascript, */*; q=0.01',
        'accept-encoding':'gzip, deflate, br',
        'accept-language':'zh-CN,zh;q=0.9',
        'cookie':'BIDUPSID=82C34C48BC2C1D5DA23220AFC1C11D69; PSTM=1562656573; BDUSS=HA2ZzdSYVpkREFqcS13NDY4T0g5fkdUUXUyUGhXaGlrdDl0OURGbnlNZWFJaXBnRVFBQUFBJCQAAAAAAAAAAAEAAABRLlQJMzA0OTExMTUAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAJqVAmCalQJgb; BDUSS_BFESS=HA2ZzdSYVpkREFqcS13NDY4T0g5fkdUUXUyUGhXaGlrdDl0OURGbnlNZWFJaXBnRVFBQUFBJCQAAAAAAAAAAAEAAABRLlQJMzA0OTExMTUAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAJqVAmCalQJgb; __yjs_duid=1_23044b93b11e8624386093f1139fbc401618040266190; Hm_lvt_aec699bb6442ba076c8981c6dc490771=1629790105; BAIDUID=DF334FF63C3732F0F6EECA0354AF72F6:FG=1; BAIDUID_BFESS=DF334FF63C3732F0F6EECA0354AF72F6:FG=1; sugstore=0; BD_HOME=1; H_PS_PSSID=35636_35104_31254_35605_34584_35491_35689_35644_35318_26350_35478; BD_UPN=12314753; delPer=0; BD_CK_SAM=1; PSINO=5; BDORZ=B490B5EBF6F3CD402E515D22BCDA1598; baikeVisitId=989dde92-7bde-464f-8586-a2680b1eadb5; COOKIE_SESSION=13_1_6_4_4_2_0_0_6_2_0_0_11309363_0_0_0_1629790110_1629790106_1640884897|9#0_1_1629790092|1; H_PS_645EC=d57dK72AujW/mvvV/zw/nKNNTtIH6+Hhc0Hv3EhdM+nAbWYHOKPXcFl6m3Y; BA_HECTOR=8h0kag0580048g2hbb1gsrql20q'
        },
        {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36',
        'accept':'application/json, text/javascript, */*; q=0.01',
        'accept-encoding':'gzip, deflate, br',
        'accept-language':'zh-CN,zh;q=0.9',
        'cookie':'BIDUPSID=2806AB9D3577A20F1291141A40A433B6; PSTM=1539695730; BD_UPN=12314753; BDUSS=BaRE5GfnpaYUlabmhLQ3hWT3B-T2d3QWdtTWNWbGprbmRvT2tXVjVkRFVXNmxmSVFBQUFBJCQAAAAAAAAAAAEAAAAvjQhsRXVydXM1MTEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAANTOgV~UzoFfaT; __yjs_duid=1_bcfa109e4f513a1d5c06db1433f59cce1619712632146; MCITY=-358%3A; BAIDUID=1309F011CED276B13DB5CA747DB72D3C:FG=1; BDORZ=FFFB88E999055A3F8A630C64834BD6D0; B64_BOT=1; BDRCVFR[FdgGNgjrejn]=mk3SLVN4HKm; delPer=0; BD_CK_SAM=1; ispeed_lsm=3; COOKIE_SESSION=13_0_8_8_0_12_1_5_3_8_3_0_0_0_0_0_0_0_1640883175%7C9%23154560_11_1637906271%7C9; PSINO=5; H_PS_645EC=f0bbYgoiwG9suueoEmDRmagtKXd0iDcuzAPVYYN6siPTSdYZb4LiHjQAbNX1ziS719A3rX14%2F%2Fxd; BD_HOME=1; H_PS_PSSID=35414_35468_35105_31253_35631_35488_34584_35491_35696_35317_26350_35617_35562; sug=3; sugstore=0; ORIGIN=0; bdime=0; BA_HECTOR=a52k0l0401242g04di1gstelo0q'
        }]
        # 构造程序文件夹
        self.father_dir = "SearchFile"
        # 手动构造搜索的父文件夹
        folder = os.path.exists(self.father_dir)
        if not folder:
            os.makedirs(self.father_dir)
            
    def mkdir(self,path):
        """
        查看某个路径是否存在，不存在的话构造文件夹
        """
        folder = os.path.exists(path)
        if not folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
            os.makedirs(path)            #makedirs 创建文件时如果路径不存在会创建这个路径
        
    def get_pageurl(self,url):
        """
        得到当前页面下所有的urls
        """
        soup = BeautifulSoup(self.url)
        print(soup)
        
    
    def search(self, word, page):
        """
        在百度浏览器中搜索word
        """
        # 页面数
        self.pagenum = page
        # 查找的关键词
        self.search_word = word
        # 对关键词中中文转码
        keyword = urllib.request.quote(self.search_word)
        # 构造url
        self.url = self.base_url + keyword
        # 通过request()封装request请求
        req = urllib.request.Request(self.url)
        # 请求网络地址得到html网页代码
        response = urllib.request.urlopen(req)
        # 以"二进制写入"模式打开问题文件夹（子文件夹） 
        self.mkdir(self.father_dir + "/" + self.search_word)
        fh = open(self.father_dir + "/" + self.search_word + "/page.html","wb")
        # 写入返回响应页面
        fh.write(response.read())
        
    def search_all(self,word,page):
        """
        在百度浏览器搜索word后爬取指定页数内的所有url
        """
        urls=[]
        #构造初始url 建立文件夹等步骤
        self.search(word,page)
        #所有初始页的url
        for num in range(0,self.pagenum):
            #按照页面数进行修改
            url_temp = self.url + "&pn=" + str(num * 10)
            urls.append(url_temp)
        print(urls)
        for url in urls:
            print(urls.index(url))
            req=requests.get(url, headers=self.headers[urls.index(url)%3], verify=False,timeout=5)
            #req.encoding='utf-8'
            req.encoding=req.apparent_encoding
            soup=BeautifulSoup(req.content)
            linkElems = soup.select('h3 > a')
            for i in range(0,len(linkElems)):
                self.pageurl.append(linkElems[i].get('href'))
            time.sleep(1)
            
            
        filename=self.father_dir + "/"+ word + str(page) + '.json'
        with open(filename,'w',encoding='utf-8') as file_obj:
            json.dump(self.pageurl,file_obj,ensure_ascii=False,indent = 4)
    
    def load_all(self,word,page,filename):
         # 页面数
        self.pagenum = page
        # 查找的关键词
        self.search_word = word
        # 对关键词中中文转码
        keyword = urllib.request.quote(self.search_word)
        # 构造url
        self.url = self.base_url + keyword
        #读取json中爬取的数据
        with open(filename,encoding="utf-8") as file_obj:
            urls=json.load(file_obj)
        self.pageurl=urls
        
        
    def download_all(self):
        contents={}
        contents_temp={}
        filename=self.father_dir + "/"+ self.search_word + str(self.pagenum) +'temp.json'
        for url in self.pageurl:
            print(len(contents))
            contents[url],contents_temp[url]=self.download_content(url)
            if len(contents)%10==0:
                with open(filename,'w',encoding='utf-8') as file_obj:
                    json.dump(contents_temp,file_obj,ensure_ascii=False,indent = 4)
            time.sleep(5)
        with open(filename,'w',encoding='utf-8') as file_obj:
            json.dump(contents_temp,file_obj,ensure_ascii=False,indent = 4)
            
        return(contents)
        
        
    def download_content(self,url):
        """
        爬取url里的文字内容
        """
        try:
            #random.choice
            req=requests.get(url, headers=random.choice(self.headers), verify=False,timeout=5)
            req.encoding=req.apparent_encoding
            soup=BeautifulSoup(req.text)
            if soup.title.string=='百度安全验证':
                return(0,0)
            return(soup,soup.getText())
        except:#网址有错误的话
            return(0,0)
        
      
            
    def check_contain_chinese(self,check_str):
        """
        检查字符串中是否包含中文字符
        """
        for ch in check_str:
            if u'\u4e00' <= ch <= u'\u9fff':
                return True
        return False
    
    def checkChinese(self,char):
        """
        判断某个字符是不是中文
        """
        if u'\u4e00' <= char <= u'\u9fff':
            return True
        return False

    def first_wash(self,contents,threshold=3):
        wash_result={}
        for url in contents.keys():
            soup=contents[url]
            text = soup.getText()
        
            i = 0
            j = 0
            length =len(text)
            dirty_position = []
            while i < length and j < length:
                while i < length:
                    if self.checkChinese(text[i]):
                        i = i + 1
                    else:
                        j = i
                        break
        
        
                while j < length:
                    if not self.checkChinese(text[j]):
                        j = j + 1 
                    else:
                        break

                if ((j-1)-i)*2 >= threshold:# 如果脏片段长度大于3，认为该片段确实是脏句子
                #if len(repr(text[i:j]))-2>=threshold:
                    dirty_position.append((i,j-1))
                elif i==length:
                    break
                i = j
            
            clean_corpus = []
            for i in range(len(dirty_position)-1):
                p_idx = dirty_position[i][1] + 1
                l_idx = dirty_position[i+1][0]
            
                clean_corpus.append(text[p_idx:l_idx])
            clean_corpus.append(text[dirty_position[-1][1]+1:])
            wash_result[url]=clean_corpus

        filename=self.father_dir + "/"+ self.search_word + str(self.pagenum) +'content.json'
        with open(filename,'w',encoding='utf-8') as file_obj:
            json.dump(wash_result,file_obj,ensure_ascii=False,indent = 4)

        return(wash_result)

In [17]:
#加一个input url的函数 然后把现有函数改成包含input url函数的函数 这样既可以直接用类里的url爬 也可以从外部提供爬
#然后把爬的部分改成 遇到百度安全认证就在url后面标0表示爬取失败

In [3]:
obj = WebSpider()

In [4]:
obj.load_all("智慧博物馆",75,"C:\\Users\\yuki\\Desktop\\spiderrrr\\spiderrrr\\SearchFile\\智慧博物馆75.json")

In [5]:
contents=obj.download_all()

0


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baike.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


1


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baijiahao.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wappass.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


2


...

745


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sohu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


746


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.sohu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


747


In [11]:
#二次爬 把一开始爬不出来的爬出来
with open("C:\\Users\\yuki\\Desktop\\spiderrrr\\spiderrrr\\SearchFile\\智慧博物馆75temp.json",encoding="utf-8") as file_obj:
    contents_temp=json.load(file_obj)
    
count=0
num=0
for i in contents_temp:
    if contents_temp[i]==0:
        num+=1
while count<=5 and num>50:
    count+=1
    for i in contents_temp:
        if contents_temp[i]==0:
            contents[i],temp=obj.download_content(i)
            contents_temp[i]=temp
            time.sleep(5)
            print(list(contents_temp.keys()).index(i))
    num=0
    for i in contents_temp:
        if contents_temp[i]==0:
            num+=1

    time.sleep(120)

D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baijiahao.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wappass.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


1


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baijiahao.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wappass.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


2


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baijiahao.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wappass.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


...

730


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baijiahao.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wappass.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


739


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baijiahao.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wappass.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


743


In [12]:
with open("C:\\Users\\yuki\\Desktop\\spiderrrr\\spiderrrr\\SearchFile\\智慧博物馆75temp2.json",'w',encoding='utf-8') as file_obj:
    json.dump(contents_temp,file_obj,ensure_ascii=False,indent = 4)

In [3]:
obj2 = WebSpider()

In [4]:
obj2.search_all("智能家具",75)

['http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%B7&pn=0', 'http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%B7&pn=10', 'http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%B7&pn=20', 'http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%B7&pn=30', 'http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%B7&pn=40', 'http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%B7&pn=50', 'http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%B7&pn=60', 'http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%B7&pn=70', 'http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%B7&pn=80', 'http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%B7&pn=90', 'http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%B7&pn=100', 'http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%B7&pn=110', 'http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%B7&pn=120', 'http://www.baidu.com/s?wd=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%85%

D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wappass.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wappass.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


71
72
73


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wappass.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


74


In [6]:
contents=obj2.download_all()

0


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'b2b.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


1
2


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baike.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


3
4
5


...

D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baike.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


716


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baike.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


717


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baike.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [8]:
#二次爬 把一开始爬不出来的爬出来
with open("C:\\Users\\yuki\\Desktop\\spiderrrr\\spiderrrr\\SearchFile\\智能家具75temp.json",encoding="utf-8") as file_obj:
    contents_temp=json.load(file_obj)
    
count=0
num=0
for i in contents_temp:
    if contents_temp[i]==0:
        num+=1
while count<=5 and num>50:
    count+=1
    for i in contents_temp:
        if contents_temp[i]==0:
            contents[i],temp=obj2.download_content(i)
            contents_temp[i]=temp
            time.sleep(5)
            print(list(contents_temp.keys()).index(i))
    num=0
    for i in contents_temp:
        if contents_temp[i]==0:
            num+=1

    time.sleep(120)

3
4
40
84
98
106
115


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.arteor.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


122


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baijiahao.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wappass.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


129
135


...

707


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baijiahao.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wappass.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


710


D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'baijiahao.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
D:\Anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wappass.baidu.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


711


In [11]:
with open("C:\\Users\\yuki\\Desktop\\spiderrrr\\spiderrrr\\SearchFile\\智能家具75temp2.json",'w',encoding='utf-8') as file_obj:
    json.dump(contents_temp,file_obj,ensure_ascii=False,indent = 4)